# Loading data from preprocessed pickle file

In [3]:
import pickle

# open the pickle file in read-binary mode
with open('..//model-data/preprocessing/train-val-sets/train_val_split.pkl', 'rb') as f:
    data = pickle.load(f)

# storing data
images_train = data['images_train']
images_val = data['images_val']
labels_train = data['labels_train']
labels_val = data['labels_val']

# Creating model

In [12]:
import tensorflow as tf
from keras import layers, Sequential

model = keras.Sequential([
    layers.Input(shape=(256, 256, 3)),
    layers.Flatten(),
    layers.Dense(39, activation='softmax')
])

# Compling the model

In [13]:
# compiling the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(images_train, labels_train, epochs=5)